<a href="https://colab.research.google.com/github/KRiver28/KRiver28/blob/master/chatbot_data_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import os

In [32]:
path = "/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/"
#파일 불러오기
df1 = pd.read_csv(path + "카페_validation.csv")
df2 = pd.read_csv(path + "카페_train.csv")
os.getcwd()

'/content'

In [33]:
#c-a , s-q 지우기
no_data1 = df1[ (df1['발화자'] == 's') & (df1['QA여부'] == 'q')].index
no_data2 = df1[ (df1['발화자'] == 'c') & (df1['QA여부'] == 'a')].index
df1_dr = df1.drop(no_data1)


no_data3 = df2[ (df2['발화자'] == 's') & (df2['QA여부'] == 'q')].index
no_data4 = df2[ (df2['발화자'] == 'c') & (df2['QA여부'] == 'a')].index
df2_dr = df2.drop(no_data3)

df1_dr.drop(no_data2, inplace = True)
df1_dr.reset_index(inplace = True, drop = True)

df2_dr.drop(no_data4, inplace = True)
df2_dr.reset_index(inplace = True, drop = True)


In [34]:
# qa 정렬대로 뽑아오기 
Q = []
A = []
Intent = []

for i in df1_dr.index:
  if df1_dr['QA여부'][i] == 'q' and df1_dr['QA여부'][i+1] == 'a':
    Q.append(df1_dr.loc[i]['발화문'])
    A.append(df1_dr.loc[i+1]['발화문'])
    Intent.append(df1_dr.loc[i]['인텐트'])

cafe_test_data= pd.DataFrame({'Q':Q,
                'A':A,
                'label':Intent})

In [35]:
# qa 정렬대로 뽑아오기
Q = []
A = []
Intent = []

for i in df2_dr.index:
  if df2_dr['QA여부'][i] == 'q' and df2_dr['QA여부'][i+1] == 'a':
    Q.append(df2_dr.loc[i]['발화문'])
    A.append(df2_dr.loc[i+1]['발화문'])
    Intent.append(df2_dr.loc[i]['인텐트'])

data= pd.DataFrame({'Q':Q,
                'A':A,
                'label':Intent})

#데이터 삭제

In [36]:
# 중복 질문 제거
# -------------------------------------------
# 데이터: 72559 --> 중복 제거 후 데이터: 66041 
data.drop_duplicates(['Q'], inplace = True)
data.reset_index(inplace = True, drop = True)

In [37]:
# label 삭제: 카페에 필요한 데이터만 남기기
# 매장, 멤버십, 예약, 웹사이트, AS, 제품_추천, 제품_용도, 비교, 재고, 교환, 제품_가격 삭제
# -------------------------------------------

del_label = data[data['label'].str.contains('매장|멤버십|예약|웹사이트|AS|제품_추천|제품_용도|비교|재고|교환|제품_가격')].index
data = data.drop(del_label)

data.reset_index(inplace = True, drop = True)

In [38]:
# -------------------------------------------
# 필요 없는 데이터 삭제
# -------------------------------------------

# 배달/배송 관련 정보 삭제
delivery_q = data[data['Q'].str.contains('배달|택배')].index
data = data.drop(delivery_q)

delivery_a = data[data['A'].str.contains('배달|택배')].index
data = data.drop(delivery_a)

delivery = data[data['label'].str.contains('배송')].index
data = data.drop(delivery)

In [39]:
# 수거 삭제
waste_q = data[data['Q'].str.contains('수거')].index
data = data.drop(waste_q)

waste_a = data[data['A'].str.contains('수거')].index
data = data.drop(waste_a)

In [40]:
# 특정 상표명, 카페 데이터 삭제
extra_q = data[data['Q'].str.contains('설빙|베스킨라빈스|스타벅스|공차|나무|단청|도마|그림|파인트|쿼터|패밀리|하프갤런|하프갤론|텀블러')].index
data = data.drop(extra_q)

extra_a = data[data['A'].str.contains('설빙|베스킨라빈스|스타벅스|공차|나무|단청|도마|그림|파인트|쿼터|패밀리|하프갤런|하프갤론|텀블러')].index
data = data.drop(extra_a)

In [41]:
# 기타 특수 데이터 삭제
dummy = data[data['A'].str.contains('오류')].index
data = data.drop(dummy)

bill = data[data['Q'].str.contains('영수증 버|영수증은 버|영수증버|영수증은버')].index
data = data.drop(bill)

price = data[data['Q'].str.contains('얼마') & data['label'].str.contains('결제_일반')].index
data = data.drop(price)

nutrient = data[data['Q'].str.contains('영양성분')].index
data = data.drop(nutrient)

nickname = data[data['A'].str.contains('닉네임')].index
data = data.drop(nickname)

etc_q = data[data['Q'].str.contains('불면증|데이터|오늘의 커피')].index
data = data.drop(etc_q)
etc_a = data[data['A'].str.contains('서비스')].index
data = data.drop(etc_a)

data.reset_index(inplace=True, drop=True)

In [42]:
# -------------------------------------------
# 답변할 수 없는 데이터 삭제
# -------------------------------------------

# 날짜 묻는 질문/날짜로 대답하는 답변 삭제
product_date_q = data[data['Q'].str.contains('1월|2월|3월|4월|5월|6월|7월|8월|9월|10월')].index
data = data.drop(product_date_q)

product_date_a = data[data['A'].str.contains('1월|2월|3월|4월|5월|6월|7월|8월|9월|10월')].index
data = data.drop(product_date_a)


# 시간 관련 질문/답변 삭제
time_q = data[data['label'].str.contains('제품_정보') & data['Q'].str.contains('몇 분|시간|몇분')].index
data = data.drop(time_q)

time_a = data[data['label'].str.contains('제품_정보') & data['A'].str.contains('몇 분|시간|몇분')].index
data = data.drop(time_a)

# 총 ~에요? 들어가는 질문 삭제
total_q = data[data['Q'].str.contains('총 얼마예요?|총 얼마')].index
data = data.drop(total_q)

total_sum = data[data['Q'].str.contains('총 몇')].index
data = data.drop(total_sum)


# 금액 안내 답변 삭제(카페에 맞게끔 새로운 데이터 추가 필요)
price_answer = data[data['A'].str.contains('원입니다|원 입니다')].index
data = data.drop(price_answer)


# 원두 삭제
coffee_q = data[data['Q'].str.contains('원두')].index
data = data.drop(coffee_q)

coffee_a = data[data['A'].str.contains('원두')].index
data = data.drop(coffee_a)


data.reset_index(inplace=True, drop=True)

In [43]:
# -------------------------------------------
# 현재 카페에 맞게끔 데이터 삭제
# -------------------------------------------

# 품절 삭제
absence_q = data[data['Q'].str.contains('품절')].index
data = data.drop(absence_q)

absence_a = data[data['A'].str.contains('품절')].index
data = data.drop(absence_a)



# 세트 메뉴 삭제
setmenu_q = data[data['Q'].str.contains('세트')].index
data = data.drop(setmenu_q)

setmenu_a = data[data['A'].str.contains('세트')].index
data = data.drop(setmenu_a)

data.reset_index(inplace=True, drop=True)

In [44]:
# 신메뉴 삭제
new_menu = data[data['label'].str.contains('제품_정보') & data['Q'].str.contains('신메뉴|새로')].index
data = data.drop(new_menu)

In [45]:
# ===========================================
# #############   데이터 학습   ###############
# ===========================================
# -------------------------------------------
# 결제 수단 학습
# 
# 가능: 현금, 카드, 모바일 페이
# 카운터 문의: 상품권, 분할 결제
# 불가: 포인트, 기프티콘, 할인
# -------------------------------------------

data.loc[data['label'].str.contains('결제_수단') & data['Q'].str.contains('페이') & data['Q'].str.contains('가능'), 'A'] = '네, 모든 모바일 페이를 지원하고 있습니다.'
data.loc[data['Q'].str.contains('어떤') & data['Q'].str.contains('페이'), 'A'] = '모든 모바일 페이를 지원하고 있습니다.'

data.loc[data['Q'].str.contains('상품권'), 'A'] = '상품권 사용은 카운터에 문의 바랍니다.'

data.loc[data['Q'].str.contains('포인트'), 'A'] = '현재 포인트 기능은 제공되지 않습니다.'

data.loc[data['Q'].str.contains('기프티콘|기포티콘|쿠폰'), 'A'] = '저희 카페는 쿠폰과 기프티콘이 없습니다.'

# 분할 결제 불가 안내 문구 학습
data.loc[data['Q'].str.contains('분할 결제|분할결제|나눠서 결제|따로 결제'), 'A'] = '분할 결제는 카운터에 문의 바랍니다.'

# 할인 불가 안내 문구 학습
data.loc[data['Q'].str.contains('할인|깎|깍|제휴|DC'), 'A'] = '현재 할인은 불가합니다.'

In [46]:
# -------------------------------------------
# 기타 학습
# 
# 카운터 문의: 오류/불량, 칼로리 정보
# 불가: 재료 변경, 디카페인
# 등
# -------------------------------------------

# 재료 변경 안내 학습
data.loc[data['Q'].str.contains('빼|교환|바꾸|추가|바꿔|뺄|변경|넣지 말|넣지 마|많이|말고|반만|반 만|조금만 더|더 넣어|올려') & data['label'].str.contains('제품_구성|제품_소재|주문_제품'), 'A'] = '구성 관련 요청은 카운터에 문의 바랍니다.'
data.loc[data['label'].str.contains('제품_정보') & data['Q'].str.contains('토핑추가|토핑 추가'), 'A'] = '구성 관련 요청은 카운터에 문의 바랍니다.'
data.loc[data['Q'].str.contains('두유') & data['label'].str.contains('제품_구성|제품_소재'), 'A'] = '현재 두유는 제공되지 않습니다.'
data.loc[data['Q'].str.contains('저지방 우유|저지방우유|무지방우유|무지방 우유'), 'A'] = '현재 저지방/무지방 우유는 제공되지 않습니다.'
data.loc[data['Q'].str.contains('당도'), 'A'] = '당도 단계 선택은 불가능합니다.'
data.loc[data['Q'].str.contains('샷 추가|샷추가') & data['label'].str.contains('제품_구성') & data['Q'].str.contains('나요|죠|가요|까요|해요|돼요'), 'A'] = '샷 추가 가능합니다.'
data.loc[data['Q'].str.contains('샷 추가|샷추가') & data['label'].str.contains('제품_구성') & ~data['Q'].str.contains('나요|죠|가요|까요|해요|돼요'), 'A'] = '샷 추가 해드리겠습니다.'

# 사이즈 관련 답변 학습(현재는 사이즈 1개로 통일)
data.loc[data['Q'].str.contains('사이즈') & ~data['Q'].str.contains('스몰|톨'), 'A'] = '저희 카페는 기본 사이즈인 톨 사이즈로만 제공하고 있습니다.'
data.loc[data['label'].str.contains('제품_정보_질문') & data['Q'].str.contains('그란데|벤티'), 'A'] = '저희 카페는 기본 사이즈인 톨 사이즈로만 제공하고 있습니다.'

# 제품 함량 문의 안내 학습
data.loc[data['Q'].str.contains('함량'), 'A'] = '상세 정보는 카운터에 문의 바랍니다.'
data.loc[data['label'].str.contains('제품_정보') & data['Q'].str.contains('그램'), 'A'] = '상세 정보는 카운터에 문의 바랍니다.'

# 디카페인 불가 안내, 카페인 문의 안내 문구 학습
data.loc[data['Q'].str.contains('디카페인|디 카페인'), 'A'] = '현재 디카페인 음료는 제공되지 않습니다.'
data.loc[data['Q'].str.contains('카페인 함량'), 'A'] = '카페인 함량은 카운터에 문의 바랍니다.'

# 리필 안내 문구 학습
data.loc[data['Q'].str.contains('리필') & data['Q'].str.contains('물|얼음'), 'A'] = '물과 얼음 리필은 카운터에 문의해주세요₍՞◌′ᵕ‵ू◌₎♡'
data.loc[data['Q'].str.contains('리필') & ~data['Q'].str.contains('물|얼음'), 'A'] = '리필은 불가능합니다. 새로 주문해주세요 ʕ”̮ॽु⋆⁺₊⋆ ♡̷̷̷ ' 



# 오류/불량 문의 안내 문구 학습
data.loc[data['label'].str.contains('제품_불량|결제_오류|주문_오류'), 'A'] = '죄송합니다. 카운터에서 확인 도와드리겠습니다.'

# 칼로리 정보 불가능 안내 문구 학습
data.loc[data['Q'].str.contains('열량|칼로리'), 'A'] = '칼로리 정보는 카운터에 문의 바랍니다.'



# 행사 안내 문구 학습
data.loc[data['Q'].str.contains('행사|이벤트'), 'A'] = '현재 진행중인 행사가 없습니다. 다음 행사를 기다려주세요₍՞◌′ᵕ‵ू◌₎♡'
data.loc[data['label'].str.contains('행사'), 'A'] = '현재 진행중인 행사가 없습니다. 다음 행사를 기다려주세요₍՞◌′ᵕ‵ू◌₎♡'

# 포장 학습
data.loc[data['label'].str.contains('포장_비용'), 'A'] = '포장 비용은 추가되지 않습니다.'

In [47]:
data

,Q,A,label
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문
...,...,...,...
33858,기프티콘으로 결제 가능한 거 맞죠?,저희 카페는 쿠폰과 기프티콘이 없습니다.,결제_수단_확인
33859,카카오페이로 결제 된거 맞죠?,네 카카오페이로 결제되셨습니다.,결제_수단_확인
33860,모바일상품권도 결제할 수 있어요?,상품권 사용은 카운터에 문의 바랍니다.,결제_수단_확인
33861,기프티콘으로 음료 계산할 수 있는 거 맞죠?,저희 카페는 쿠폰과 기프티콘이 없습니다.,결제_수단_확인


#추가 데이터 병합

In [48]:
path = "/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/"
#QA추가 csv 파일 불러오기
df_add = pd.read_csv(path + "cafe_add.csv")
os.getcwd()

'/content'

In [49]:
df_add

,Q,A,Label
0,아메리카노 얼마에요?,아메리카노 가격은 3500원 입니다.,제품_가격_질문
1,따뜻한 아메리카노 얼마에요?,따뜻한 아메리카노 가격은 3500원 입니다.,제품_가격_질문
2,아이스 아메리카노 얼마에요?,아이스 아메리카노 가격은 3500원 입니다.,제품_가격_질문
3,아메리카노 주세요,"차가운 것으로 드릴까요?, 따뜻한 것으로 드릴까요?",주문_제품_요청
4,아메리카노 한 잔 주세요,"차가운 것으로 드릴까요?, 따뜻한 것으로 드릴까요?",주문_제품_요청
...,...,...,...
2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",제품_정보_질문
2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",주문_제품_확인
2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",주문_제품_확인
2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,주문_제품_확인


In [50]:
df_comb = pd.concat([data, df_add])

In [51]:
df_comb

,Q,A,label,Label
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문,NaN
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문,NaN
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문,NaN
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문,NaN
4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,제품_불량_질문,NaN
...,...,...,...,...
2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",NaN,제품_정보_질문
2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",NaN,주문_제품_확인
2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",NaN,주문_제품_확인
2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,NaN,주문_제품_확인


In [52]:
#################################
### label 항목 레이블, 원핫인코딩(의도분석에 이용)
#################################

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#레이블 인코딩
lab_encoder = LabelEncoder()
lab_encoder.fit(df_comb['label'].unique())
lst = lab_encoder.classes_
label_list = dict(zip(lst, range(len(lst))))

print(label_list)    #원래값-숫자 목록

{'결제_방식_요청': 0, '결제_방식_질문': 1, '결제_방식_확인': 2, '결제_수단_요청': 3, '결제_수단_질문': 4, '결제_수단_확인': 5, '결제_시기_요청': 6, '결제_시기_질문': 7, '결제_시기_확인': 8, '결제_영수증_요청': 9, '결제_영수증_질문': 10, '결제_영수증_확인': 11, '결제_오류_요청': 12, '결제_오류_질문': 13, '결제_오류_확인': 14, '결제_일반_요청': 15, '결제_일반_질문': 16, '결제_일반_확인': 17, '결제_재결제_요청': 18, '결제_재결제_질문': 19, '결제_재결제_확인': 20, '결제_추가_요청': 21, '결제_추가_질문': 22, '결제_추가_확인': 23, '결제_취소_요청': 24, '결제_취소_질문': 25, '결제_취소_확인': 26, '결제_할인_요청': 27, '결제_할인_질문': 28, '결제_할인_확인': 29, '제품_구성_요청': 30, '제품_구성_질문': 31, '제품_구성_확인': 32, '제품_날짜_요청': 33, '제품_날짜_질문': 34, '제품_날짜_확인': 35, '제품_방법_요청': 36, '제품_방법_질문': 37, '제품_방법_확인': 38, '제품_불량_요청': 39, '제품_불량_질문': 40, '제품_불량_확인': 41, '제품_소재_요청': 42, '제품_소재_질문': 43, '제품_소재_확인': 44, '제품_원산지_요청': 45, '제품_원산지_질문': 46, '제품_원산지_확인': 47, '제품_일반_요청': 48, '제품_일반_질문': 49, '제품_일반_확인': 50, '제품_입고_요청': 51, '제품_입고_질문': 52, '제품_입고_확인': 53, '제품_정보_요청': 54, '제품_정보_질문': 55, '제품_정보_확인': 56, '제품_품질_요청': 57, '제품_품질_질문': 58, '제품_품질_확인': 59, '주문_변경_요청': 60, '주문_변경_질문': 61, '주문_변경_확

In [53]:
#레이블 데이터 적용
lab_in=lab_encoder.transform(df_comb['label'])   # label 값
ori_in=lab_encoder.inverse_transform(lab_in)         # 원래 값
df_comb['label'] = lab_in

In [54]:
df_comb

,Q,A,label,Label
0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
...,...,...,...,...
2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",96,제품_정보_질문
2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",96,주문_제품_확인
2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",96,주문_제품_확인
2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,96,주문_제품_확인


#데이터 저장

In [55]:
df_comb.to_csv(path + 'ChatbotData.csv')

In [56]:
#koGPT2 위치 저장
df_comb.to_csv('/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/KoGPT2-chatbot-master/koGPT2_test/Chatbot_data/ChatbotData.csv')

In [57]:
pd.read_csv(path +'ChatbotData.csv')

,Unnamed: 0,Q,A,label,Label
0,0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
1,1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
2,2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
3,3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
4,4,시루떡에서 왜 시큼한 냄새가 나는 거죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,40,NaN
...,...,...,...,...,...
35993,2164,마실 것 뭐가 있나요?,"음료 메뉴는 아메리카노, 카페라떼, 카페모카, 바닐라라떼, 콜드브루, 카라멜 마끼아...",96,제품_정보_질문
35994,2165,안녕하세요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",96,주문_제품_확인
35995,2166,저기요,"어서오세요, Learning Crew Café 입니다. 무엇을 도와드릴까요?",96,주문_제품_확인
35996,2167,벌레가 나왔어요,죄송합니다. 카운터에서 확인 도와드리겠습니다.,96,주문_제품_확인


In [58]:
cafe_test_data.to_csv(path + 'cafe_qa_validation.csv')

In [59]:
pd.read_csv(path +'cafe_qa_validation.csv')

,Unnamed: 0,Q,A,label
0,0,케이크에서 약간 시큼한 맛이 나는데 상한건가요?,"죄송합니다, 새걸로 바꿔드릴게요.",제품_불량_질문
1,1,주스 안에서 돌 같은게 나왔는데 확인 좀 해주시겠어요?,제조과정에서 문제가 있었던 걸로 추측되네요.,제품_불량_질문
2,2,이 에그타르트 가운데가 너무 날계란물인 것 같은데 원래 이런 거예요?,안쪽이 약간 덜 익은 제품이 나갔나봐요. 다른 걸로 드릴게요.,제품_불량_질문
3,3,이 딸기 케이크 사이즈가 잘못 표기된 것 같은데요.,바로 확인해서 알려드리겠습니다.,제품_불량_질문
4,4,케이크 주문했는데 생크림 상한 것 같습니다.,죄송합니다. 환불을 원하시면 바로 환불해 드릴게요.,제품_불량_질문
...,...,...,...,...
9517,9517,포인트 차감으로 프레첼 머랭 쿠키 구매할 수 있는 거죠?,"네, 포인트 차감으로 프레첼 머랭 쿠키를 구매하실 수 있습니다.",결제_수단_확인
9518,9518,지난주에 매장에서 보내준 생일 쿠폰이 있어요.,"네, 1년 안에 쿠폰 쓰시면 됩니다.",결제_수단_확인
9519,9519,카카오톡 기프티콘이랑 롯데 상품권 동시에 사용해도 되는 거죠?,"네, 기프티콘과 상품권 동시 사용 가능합니다.",결제_수단_확인
9520,9520,기프티콘 적용된 것 맞나요?,네. 잘 적용되었습니다.,결제_수단_확인
